In [ ]:
#やりたい
#①resultオブジェクトをdatetimeやstring形式に変換可能か？(max, sortで取得したデータはすべてresult -> 比較できない)
#　　result型：query(Date.XXX)やquert(func.max)など query内での条件絞りはNG
#   ◎：抜き出したデータをlistに格納すればOK
#②表の１行目に行を追加する->　無理っぽい

In [17]:
#１ページ目のデータ取得し、現在DBになければ追加する ※休日は更新がないため

#1ページ目のデータ取得
from selenium import webdriver
browser = webdriver.Chrome('chromedriver.exe')

url = 'https://minkabu.jp/stock/100000018/daily_bar'
browser.get(url)

#要素抽出
elems = browser.find_element_by_id('fourvalue_timeline')
elems = elems.find_elements_by_tag_name('tr')

dates = []
starts = []
highs = []
lows = []
ends = []
adjusteds = []


#取得データを各listに入れる *0行目はタイトルのため不要
def extract():
    for elem in elems[1:]:      #list[start:end]
        data = elem.text.split(" ")
        
        date = data[0]
        start = data[1]
        high = data[2]
        low = data[3]
        end = data[4]
        adjusted = data[5]

        dates.append(date)
        starts.append(start)
        highs.append(high)
        lows.append(low)
        ends.append(end)
        adjusteds.append(adjusted)

extract()

In [100]:
dates[0]

'2020/02/10'

In [183]:
#取得した日付をstr->dateにする(先頭＝最新日のみ)
import datetime
scraypedDate = datetime.datetime.strptime(dates[0],'%Y/%m/%d').date()
scraypedDate

datetime.date(2020, 2, 10)

In [107]:
#dateの値を一つづつ取り出し、既存の表の最新日と一致していなければrowを作りDBに追加
#DBの読込み
from assets.database import db_session
from assets.models import Data
import datetime

In [177]:
dbDateTop = db_session.query(Data).all()[0].date
dbDateTop

2020-02-12 07:54:40,996 INFO sqlalchemy.engine.base.Engine SELECT data.id AS data_id, data.date AS data_date, data.start AS data_start, data.high AS data_high, data.low AS data_low, data."end" AS data_end, data.adjusted AS data_adjusted, data.timestamp AS data_timestamp 
FROM data
2020-02-12 07:54:40,997 INFO sqlalchemy.engine.base.Engine ()


datetime.date(2016, 10, 31)

In [88]:
# #sqlAlchemy　funcのインポート
# #func.max(Data.列タイトル)で最大値を抜き出せるが、Resultオブジェクトのため使いまわしができない　-> func系は使えない
# from sqlalchemy import func
# latestDate = db_session.query(func.max(Data.date))[0]
# latestDate

2020-02-12 00:54:35,425 INFO sqlalchemy.engine.base.Engine SELECT max(data.date) AS max_1 
FROM data
 LIMIT ? OFFSET ?
2020-02-12 00:54:35,427 INFO sqlalchemy.engine.base.Engine (1, 0)


(datetime.date(2020, 2, 10))

In [4]:
#日付の比較
#scraypedDate:スクレイプした日付、latestDate:表の最上or最下で大きい方の日付

#DBの最新日(latestDate)
from assets.database import db_session
from assets.models import Data
import datetime


###日付を比較して、新しい日を見つける作戦。※一番下に新しい日付がくるとは限らない。ブロックで入れた場合など。→ 結局１つづつ比較するしかない
dbDates = db_session.query(Data.date).all()
dbDateTop = dbDates[0].date

#date列を１列ずつdatetime型に変換
for index, dbDate in enumerate(dbDates):
    dbDate = dbDates[index].date

    if dbDate > dbDateTop:
        latestDate = dbDate
    else:
        latestDate = dbDateTop

if scraypedDate > latestDate:

    #scraypしたデータを一列ずつrowに入れる（dateはdatetime型に変換）
    for date, start, high, low, end, adjusted in zip(dates, starts, highs, lows, ends, adjusteds):
        date = datetime.strptime(date,'%Y/%m/%d').date()

        row = Data(date = date, start = start, high = high, low = low, end = end, adjusted= adjusted)
        db_session.add(row)
    
    db_session.commit()

else:
    pass

2020-02-12 08:46:10,059 INFO sqlalchemy.engine.base.Engine SELECT data.date AS data_date 
FROM data
2020-02-12 08:46:10,060 INFO sqlalchemy.engine.base.Engine ()


In [3]:
#DBへの最新日付追加処理
#１ページ目のデータ取得し、現在DBになければ追加する ※休日は更新がないため
#取得データ最新日とDB内の最新日付と比較し、新しいデータがあればDBに追加


#1ページ目のデータ取得
from selenium import webdriver
browser = webdriver.Chrome('chromedriver.exe')

url = 'https://minkabu.jp/stock/100000018/daily_bar'
browser.get(url)

#要素抽出
elems = browser.find_element_by_id('fourvalue_timeline')
elems = elems.find_elements_by_tag_name('tr')

dates = []
starts = []
highs = []
lows = []
ends = []
adjusteds = []


#取得データを各listに入れる *0行目はタイトルのため不要
def extract():
    for elem in elems[1:]:      #list[start:end]
        data = elem.text.split(" ")
        
        date = data[0]
        start = data[1]
        high = data[2]
        low = data[3]
        end = data[4]
        adjusted = data[5]

        dates.append(date)
        starts.append(start)
        highs.append(high)
        lows.append(low)
        ends.append(end)
        adjusteds.append(adjusted)

extract()


#取得した日付をstr->dateにする(先頭＝最新日のみ)
import datetime
scraypedDate = datetime.datetime.strptime(dates[0],'%Y/%m/%d').date()
scraypedDate



#日付の比較
#scraypedDate:スクレイプした日付、latestDate:表の最上or最下で大きい方の日付

#DBの最新日(latestDate)
from assets.database import db_session
from assets.models import Data
import datetime


###日付を比較して、新しい日を見つける作戦。※一番下に新しい日付がくるとは限らない。ブロックで入れた場合など。→ 結局１つづつ比較するしかない
dbDates = db_session.query(Data.date).all()
dbDateTop = dbDates[0].date

#date列を１列ずつdatetime型に変換
for index, dbDate in enumerate(dbDates):
    dbDate = dbDates[index].date

    if dbDate > dbDateTop:
        latestDate = dbDate
    else:
        latestDate = dbDateTop

if scraypedDate > latestDate:

    #scraypしたデータを一列ずつrowに入れる（dateはdatetime型に変換）
    for date, start, high, low, end, adjusted in zip(dates, starts, highs, lows, ends, adjusteds):
        date = datetime.strptime(date,'%Y/%m/%d').date()

        row = Data(date = date, start = start, high = high, low = low, end = end, adjusted= adjusted)
        db_session.add(row)
    
    db_session.commit()

else:
    pass

2020-02-12 08:46:09,894 INFO sqlalchemy.engine.base.Engine SELECT data.date AS data_date 
FROM data
2020-02-12 08:46:09,895 INFO sqlalchemy.engine.base.Engine ()


In [6]:
#result型の処理
#sqlAlchemy　funcのインポート
#func.max(Data.列タイトル)で最大値を抜き出せるが、Resultオブジェクトのため使いまわしができない　-> func系は使えない
from sqlalchemy import func
maxDate = db_session.query(func.max(Data.date))[0]
type(maxDate)

2020-02-12 08:48:10,688 INFO sqlalchemy.engine.base.Engine SELECT max(data.date) AS max_1 
FROM data
 LIMIT ? OFFSET ?
2020-02-12 08:48:10,689 INFO sqlalchemy.engine.base.Engine (1, 0)


sqlalchemy.util._collections.result

In [10]:
maxDate

(datetime.date(2020, 2, 10))

In [20]:
from sqlalchemy import func
data = db_session.query(func.max(Data.date)) #result型

maxDates = []

for datum in data:
    maxDates.append(datum)

type(maxDates[0])

#NG。結局できていない

2020-02-12 08:52:01,231 INFO sqlalchemy.engine.base.Engine SELECT max(data.date) AS max_1 
FROM data
2020-02-12 08:52:01,233 INFO sqlalchemy.engine.base.Engine ()


sqlalchemy.util._collections.result

In [24]:
#query(Data.XXX)はrsultから一般の型に変更可能
data = db_session.query(Data.date).all() #result型

dates = []

for datum in data:
    dates.append(datum.date)

dates[0]

2020-02-12 08:55:40,518 INFO sqlalchemy.engine.base.Engine SELECT data.date AS data_date 
FROM data
2020-02-12 08:55:40,520 INFO sqlalchemy.engine.base.Engine ()


datetime.date(2020, 2, 10)